In [1]:
"""
Risk-Model Benchmark + SHAP Explanations
---------------------------------------
Everything from your original pipeline, plus:
  • shap.Tree/Linear/KernelExplainer selection
  • per-model top-10 feature importances (SHAP)
"""

import pandas as pd, numpy as np, warnings, shap, inspect, sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import StandardScaler, OneHotEncoder
from sklearn.compose        import ColumnTransformer
from sklearn.metrics        import (accuracy_score, precision_score,
                                    recall_score, f1_score, roc_auc_score)
from sklearn.ensemble       import (RandomForestClassifier,
                                    GradientBoostingClassifier,
                                    HistGradientBoostingClassifier)
from sklearn.linear_model   import LogisticRegression
from sklearn.multiclass     import OneVsRestClassifier
from sklearn.svm            import SVC
import xgboost as xgb, lightgbm as lgb
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")        # cleaner console
N_EXPL = 200                             # SHAP sample size (adjust ↑↓)

# -------------------- Load & clean --------------------
df = pd.read_csv(r"D:\jilo_new.csv", encoding="latin1")
df.drop(columns=[c for c in df.columns if "id" in c.lower()], inplace=True)

# Gender → numeric ----------------------------------------------------------
gcol = next((c for c in df.columns if "gender" in c.lower()), None)
if gcol:
    df[gcol] = df[gcol].astype(str).str.lower().str.strip()           \
                       .map({"male":1,"m":1,"female":0,"f":0})

# Targets -------------------------------------------------------------------
targets = ["Overall Risk","Cardiac Risk","Diabetic Risk","Hypertension Risk"]
bad_lbl = {"unknown","not assessed","na",""}
df = df.dropna(subset=targets)
for col in targets:
    df = df[~df[col].astype(str).str.lower().isin(bad_lbl)]

# Mode imputation -----------------------------------------------------------
for c in df.columns:
    if df[c].isnull().any():
        df[c] = df[c].fillna(df[c].mode(dropna=True)[0])

# -------------------- Preprocessing ------------------
X = df.drop(columns=targets)
cat_cols = X.select_dtypes(include="object").columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

preproc = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols)
])

# -------------------- Models -------------------------
base = {
    "LogReg"  : LogisticRegression(max_iter=1000, class_weight="balanced"),
    "RandFor" : RandomForestClassifier(n_estimators=150, max_depth=15,
                                       class_weight="balanced"),
    "XGBoost" : xgb.XGBClassifier(use_label_encoder=False,
                                  eval_metric="mlogloss", max_depth=6),
    "SVC"     : SVC(probability=True, kernel="rbf", class_weight="balanced"),
    "GradBoost": GradientBoostingClassifier(n_estimators=150, learning_rate=0.1),
    "HistGB"  : HistGradientBoostingClassifier(max_iter=150),
    "CatBoost": CatBoostClassifier(verbose=0),
    "LightGBM": lgb.LGBMClassifier()
}

# helper: pick SHAP explainer type ------------------------------------------
def get_explainer(estimator):
    """Return a SHAP explainer suited to the underlying model."""
    raw = estimator
    # unwrap OneVsRest -> take first class estimator for speed
    if hasattr(estimator, "estimators_"):
        raw = estimator.estimators_[0]
    # tree-based
    tree_types = ("XGB","LGBM","CatBoost","RandomForest","GradientBoosting",
                  "HistGradientBoosting")
    if any(t in type(raw).__name__ for t in tree_types):
        return shap.TreeExplainer(raw)
    # linear
    if isinstance(raw, LogisticRegression):
        return shap.LinearExplainer(raw, masker=shap.maskers.Independent)
    # fallback
    return shap.KernelExplainer(raw.predict_proba,
                                shap.sample(background, 100, random_state=42))

# -------------------- Evaluation + SHAP ------------------------------------
results = {}

for tgt in targets:
    print(f"\n===== {tgt} =====")
    y = df[tgt].astype("category").cat.codes
    Xtr, Xte, ytr, yte = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)

    # fit preprocessor
    Xtr_p = preproc.fit_transform(Xtr)
    Xte_p = preproc.transform(Xte)
    feat_names = preproc.get_feature_names_out()

    results[tgt] = {}

    for name, clf0 in base.items():
        model = OneVsRestClassifier(clf0)
        model.fit(Xtr_p, ytr)
        yhat  = model.predict(Xte_p)

        # metrics -----------------------------------------------------------
        try:
            yprob = model.predict_proba(Xte_p)
            auc   = roc_auc_score(yte, yprob, multi_class="ovr")
        except Exception:
            auc = np.nan

        results[tgt][name] = {
            "Accuracy" : accuracy_score(yte, yhat),
            "Precision": precision_score(yte, yhat, average="macro", zero_division=0),
            "Recall"   : recall_score(yte, yhat, average="macro", zero_division=0),
            "F1-Score" : f1_score(yte, yhat, average="macro", zero_division=0),
            "ROC AUC"  : auc
        }

        # ----------------- SHAP explanation block -------------------------
        try:
            explainer = get_explainer(model)
            # sample a manageable slice of test data
            X_sample  = shap.sample(Xte_p, min(N_EXPL, Xte_p.shape[0]),
                                    random_state=42)
            sv = explainer(X_sample, check_additivity=False)

            # mean |SHAP| per feature
            mean_abs = np.abs(sv.values).mean(axis=0)
            idx_top  = np.argsort(mean_abs)[-10:][::-1]
            print(f"\n▶ Top-10 SHAP features for {name}")
            for i in idx_top:
                print(f"   {feat_names[i]}: {mean_abs[i]:.4f}")

            # OPTIONAL: uncomment for graphics (Jupyter)
            # shap.summary_plot(sv, X_sample, feature_names=feat_names,
            #                   show=False, plot_size=(8,4))

        except Exception as e:
            print(f"(SHAP skipped for {name}: {e})")

# -------------------- Comparison tables ------------------------------------
for tgt in targets:
    print(f"\n=== {tgt} Model Comparison ===")
    (pd.DataFrame(results[tgt]).T
       .sort_values("F1-Score", ascending=False)
       .round(3)
       .pipe(print))



===== Overall Risk =====
(SHAP skipped for LogReg: The Linear explainer only supports the Independent, Partition, and Impute maskers right now!)

▶ Top-10 SHAP features for RandFor
(SHAP skipped for RandFor: unsupported format string passed to numpy.ndarray.__format__)

▶ Top-10 SHAP features for XGBoost
   num__Diastolic BP (mmHg): 1.9963
   num__Systolic BP (mmHg): 1.4280
   num__Blood Glucose (mg/dL): 1.0456
   num__RespiratoryRate(breaths/min): 0.4187
   num__Heart Rate (bpm): 0.3839
   num__Palpitation: 0.3270
   num__Diabetes_Family: 0.3229
   num__BMI: 0.2830
   num__CVD_Family: 0.2710
   num__Age: 0.2184
(SHAP skipped for SVC: name 'background' is not defined)

▶ Top-10 SHAP features for GradBoost
   num__Diastolic BP (mmHg): 2.3412
   num__Blood Glucose (mg/dL): 1.6962
   num__Systolic BP (mmHg): 1.2781
   num__BMI: 0.4117
   num__CVD_Family: 0.2729
   num__Age: 0.2609
   num__Heart Rate (bpm): 0.2140
   num__Tobacco: 0.1984
   num__RespiratoryRate(breaths/min): 0.1859
   num